In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time


In [4]:

# Initialize webdriver
driver = webdriver.Chrome()

driver.get("https://misbar.com/factcheck")

# Scraping function
def scrape_articles():
    articles = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'factcheck-item'))
    )
    scraped_data = []
    for article in articles:
        title_element = article.find_element(By.CLASS_NAME, 'card-title')
        title = title_element.text
        category = article.find_element(By.CLASS_NAME, 'category-type').text
        legitimacy = article.find_element(By.CLASS_NAME, 'classification').text
        # Map legitimacy classifications to a single category
        if legitimacy in ["صحيح", "زائف", "مفبرك", "خطأ شنيع", "حقيقي", "مزيف"]:
            if legitimacy in ["زائف", "مفبرك", "خطأ شنيع", "مزيف"]:
                legitimacy = "زائف"
            scraped_data.append([title, category, legitimacy])
    return scraped_data


In [5]:

# Function to click the "Load More" button to load more articles for scraping
def click_load_more():
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'load-more'))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
        time.sleep(3)
        # Click the button
        driver.execute_script("arguments[0].click();", load_more_button)
        return True
    except Exception:
        return False


In [6]:

# Retrieve all the category filters
filter_categories = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, 'dropdown-item'))
)

# Iterate through all categories except the first (which is a mixture of all the categories)
for category in filter_categories[1:]:
    category_text = category.text
    driver.execute_script("arguments[0].scrollIntoView(true);", category)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", category)
    
    # Wait for the first article to change to confirm the page has updated
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'factcheck-item'))
    )
    # Allow additional time for the full list to load
    time.sleep(2)
    
    # Scrape articles
    scraped_data = scrape_articles()

    # Load more articles multiple times
    for _ in range(100):
        if not click_load_more():
            break
        # Wait for more articles to load
        time.sleep(2)
        scraped_data.extend(scrape_articles())

    # Save data to a CSV file for the current category
    with open(f'{category_text}_articles.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Article', 'Topic', 'Legitimacy'])
        writer.writerows(scraped_data)

# Quit the web driver
driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.122)
Stacktrace:
	GetHandleVerifier [0x00007FF67E157032+63090]
	(No symbol) [0x00007FF67E0C2C82]
	(No symbol) [0x00007FF67DF5EC65]
	(No symbol) [0x00007FF67DF3CA7C]
	(No symbol) [0x00007FF67DFCD687]
	(No symbol) [0x00007FF67DFE2AC1]
	(No symbol) [0x00007FF67DFC6D83]
	(No symbol) [0x00007FF67DF983A8]
	(No symbol) [0x00007FF67DF99441]
	GetHandleVerifier [0x00007FF67E5525AD+4238317]
	GetHandleVerifier [0x00007FF67E58F70D+4488525]
	GetHandleVerifier [0x00007FF67E5879EF+4456495]
	GetHandleVerifier [0x00007FF67E230576+953270]
	(No symbol) [0x00007FF67E0CE54F]
	(No symbol) [0x00007FF67E0C9224]
	(No symbol) [0x00007FF67E0C935B]
	(No symbol) [0x00007FF67E0B9B94]
	BaseThreadInitThunk [0x00007FFEFC8E257D+29]
	RtlUserThreadStart [0x00007FFEFE6CAA58+40]
